У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [27]:
import warnings

warnings.filterwarnings("ignore")
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [14]:
customers_df = pd.read_csv('csv/customer_segmentation_train.csv')

customers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [2]:
customers_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [12]:
print('Null values in Ever_Married column: ', customers_df.Ever_Married.isnull().sum())
customers_df.Ever_Married.value_counts()

Null values in Ever_Married column:  140


Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64

In [6]:
print('Null values in Profession column: ', customers_df.Profession.isnull().sum())
customers_df.Profession.value_counts()

Нульові значення в колонці Profession:  124


Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64

In [7]:
print('Number of null values in Work_Experience column: ', customers_df.Work_Experience.isnull().sum())
customers_df.Work_Experience.value_counts()

Number of null values in Work_Experience column:  829


Work_Experience
1.0     2354
0.0     2318
9.0      474
8.0      463
2.0      286
3.0      255
4.0      253
6.0      204
7.0      196
5.0      194
10.0      53
11.0      50
12.0      48
13.0      46
14.0      45
Name: count, dtype: int64

In [8]:
print('Null values in Family_Size column: ', customers_df.Family_Size.isnull().sum())
customers_df.Family_Size.value_counts()

Null values in Family_Size column:  335


Family_Size
2.0    2390
3.0    1497
1.0    1453
4.0    1379
5.0     612
6.0     212
7.0      96
8.0      50
9.0      44
Name: count, dtype: int64

In [9]:
print('Null values in Var_1 column: ', customers_df.Var_1.isnull().sum())
customers_df.Var_1.value_counts()

Null values in Var_1 column:  76


Var_1
Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: count, dtype: int64

In [45]:
if 'ID' in customers_df.columns:
    customers_df.drop('ID', axis=1, inplace=True)

input_cols = list(customers_df.columns)[1:-1]
target_col = 'Segmentation'

inputs_df = customers_df[input_cols].copy()
targets_df = customers_df[target_col].copy()

numeric_cols = inputs_df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = inputs_df.select_dtypes('object').columns.tolist()

numeric_imputer = SimpleImputer(strategy='mean')
numeric_imputer.fit(inputs_df[numeric_cols])
inputs_df[numeric_cols] = numeric_imputer.transform(inputs_df[numeric_cols])

numeric_scaler = MinMaxScaler()
numeric_scaler.fit(inputs_df[numeric_cols])
inputs_df[numeric_cols] = numeric_scaler.transform(inputs_df[numeric_cols])

# 1. Fit the categorical imputer and encoder on the categorical columns:
categorical_imputer = SimpleImputer(strategy='most_frequent')
inputs_df[categorical_cols] = categorical_imputer.fit_transform(inputs_df[categorical_cols])

categorical_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_array = categorical_encoder.fit_transform(inputs_df[categorical_cols])

# 2. Create a new DataFrame from the encoded array, using the encoder’s feature names.
encoded_cat_df = pd.DataFrame(encoded_array,
                              columns=categorical_encoder.get_feature_names_out(categorical_cols),
                              index=inputs_df.index)

# 3. Drop the original categorical columns from inputs:
inputs_df = inputs_df.drop(columns=categorical_cols)

# 4. Concatenate the encoded categorical features with the remaining data:
inputs_df = pd.concat([inputs_df, encoded_cat_df], axis=1)

transformed_customers_df = pd.concat([inputs_df, targets_df], axis=1)

transformed_customers_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       8068 non-null   float64
 1   Work_Experience           8068 non-null   float64
 2   Family_Size               8068 non-null   float64
 3   Ever_Married_No           8068 non-null   float64
 4   Ever_Married_Yes          8068 non-null   float64
 5   Graduated_No              8068 non-null   float64
 6   Graduated_Yes             8068 non-null   float64
 7   Profession_Artist         8068 non-null   float64
 8   Profession_Doctor         8068 non-null   float64
 9   Profession_Engineer       8068 non-null   float64
 10  Profession_Entertainment  8068 non-null   float64
 11  Profession_Executive      8068 non-null   float64
 12  Profession_Healthcare     8068 non-null   float64
 13  Profession_Homemaker      8068 non-null   float64
 14  Professi

In [48]:
use_sample = False
train_val_df, test_df = train_test_split(transformed_customers_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(transformed_customers_df, test_size=0.2, random_state=42)

train_inputs = train_df.iloc[:, :-1].copy()
train_targets = train_df.iloc[:, -1].copy()

val_inputs = val_df.iloc[:, :-1].copy()
val_targets = val_df.iloc[:, -1].copy()

test_inputs = test_df.iloc[:, :-1].copy()
test_targets = test_df.iloc[:, -1].copy()

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [49]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

# Perform random sampling
smote = SMOTE(random_state=0)
train_inputs_sm, train_targets_sm = smote.fit_resample(train_inputs, train_targets)

smotetomek = SMOTETomek(random_state=0)
train_inputs_smt, train_targets_smt = smotetomek.fit_resample(train_inputs, train_targets)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.multiclass import OneVsRestClassifier

log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)

ovr_model.fit(train_inputs, train_targets)
ovr_predictions = ovr_model.predict(test_inputs)

print('Original dataset \n')
print(classification_report(test_targets, ovr_predictions))

log_reg = LogisticRegression(solver='liblinear')
ovr_sm_model = OneVsRestClassifier(log_reg)

ovr_sm_model.fit(train_inputs_sm, train_targets_sm)
ovr_sm_predictions = ovr_sm_model.predict(test_inputs)

print('SMOTE dataset \n')
print(classification_report(test_targets, ovr_sm_predictions))

log_reg = LogisticRegression(solver='liblinear')
ovr_smt_model = OneVsRestClassifier(log_reg)

ovr_smt_model.fit(train_inputs_smt, train_targets_smt)
ovr_smt_predictions = ovr_smt_model.predict(test_inputs)

print('SMOTE-Tomek dataset \n')
print(classification_report(test_targets, ovr_smt_predictions))

Original dataset 

              precision    recall  f1-score   support

           A       0.40      0.49      0.44       391
           B       0.36      0.13      0.19       369
           C       0.46      0.59      0.52       380
           D       0.66      0.70      0.68       474

    accuracy                           0.49      1614
   macro avg       0.47      0.48      0.46      1614
weighted avg       0.48      0.49      0.47      1614

SMOTE dataset 

              precision    recall  f1-score   support

           A       0.39      0.50      0.44       391
           B       0.39      0.21      0.27       369
           C       0.47      0.58      0.52       380
           D       0.70      0.65      0.67       474

    accuracy                           0.50      1614
   macro avg       0.49      0.49      0.48      1614
weighted avg       0.50      0.50      0.49      1614

SMOTE-Tomek dataset 

              precision    recall  f1-score   support

           A      

As we can see original dataset, SMOTE and SMOTE-Tomek datasets show similar metrics. It says that SMOTE and SMOTE-Tomek resampling did not give much improvement in my flow. The reason may be that we have not so imballanced classes. Maybe some more feature engineering may help to improve dataset results after resampling.
As a conclusion, I can say that there is no difference which model to use in this particular case.